# https://www.kaggle.com/code/akshitrai/chatbot-jarvis

In [1]:
import os
import json
import numpy as np 
import string
from unicode_tr import unicode_tr
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer
from nltk import ngrams
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline

In [2]:
path = r'C:\Users\alierenekinci\Desktop\Project\ThreadTeknofest\veri'

def yukle():
    liste = os.listdir(path)
    final_path = path + "\\" + liste[-1]
    print(final_path)
    with open(final_path, encoding='utf-8') as f:
        veri = json.load(f)
    return veri

In [4]:
df_ = yukle()

C:\Users\alierenekinci\Desktop\Project\ThreadTeknofest\veri\ankarabbSoruCevapVeriSetiJson-v12-onislenmis.json


In [5]:
questions, responses = [],[]


for veri_ in df_:
    for i, j in enumerate(df_[veri_]):
        for k,l in enumerate(j["patterns"]):
            questions.append(l)
            responses.append(df_[veri_][i]["responses"][0])
        # for k,l in enumerate(j["responses"]):
        #     responses.append(l)

In [6]:
df = pd.DataFrame([questions,responses]).T

In [ ]:
df

In [7]:
Pipe = Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier', DecisionTreeClassifier())
])

In [8]:
Pipe.fit(df[0],df[1])

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [9]:
def metin_onisle(metin):
    kucuk_harfli_metin = unicode_tr(metin).lower()
    istenen_karakterler = set(list(' abcdefghijklmnopqrstuvwxyzâçîöüğış0123456789'))
    harfler = list(kucuk_harfli_metin)
    harfler = [k if k in istenen_karakterler else ' ' for k in harfler]
    temiz_dokuman = "".join(kucuk_harfli_metin)
    temiz_dokuman_kelimeleri = temiz_dokuman.split(' ')
    temiz_dokuman_kelimeleri = [kelime for kelime in temiz_dokuman_kelimeleri if len(kelime) > 0]
    turkStem=TurkishStemmer()
    temiz_dokuman_kelimeleri = [turkStem.stemWord(kelime) for kelime in temiz_dokuman_kelimeleri]
    n = 2
    bigrams = ngrams(temiz_dokuman_kelimeleri, n)
    bigramstr = map(''.join, bigrams)
    ngram = " ".join(list(bigramstr))
    temiz_dokuman = " ".join(temiz_dokuman_kelimeleri) + " " +  "".join(ngram)
    return temiz_dokuman

In [23]:
Pipe.predict([metin_onisle("Evlendirme İşlemleri Hizmetleri bilgi")])[0]

'Evlendirme Şube Müdürlüğü evlenmek için başvuruda bulunan Türk vatandaşı ve Yabancı uyruklu kişilerin evlilik başvurularını almakta ve Gençlik Parkı içerisinde 1 adet oda nikah salonunda oda nikahı hizmeti ve Ankara mücavir alan sınırları içerinde bulunan çiftlerin belirlediği mekanlara nikah memuru göndererek nikah hizmeti vermektedir.'

In [32]:
Pipe.predict([metin_onisle("Başkent Kartın Aidat Ücreti")])[0]

'Türkiye’nin en kapsamlı şehir kartı bir çok alanda hayatınızı kolaylaştırırken, siz de herhangi bir kart aidatı ödemezsiniz.'

In [34]:
Pipe.predict([metin_onisle("Cenaze / defin işlemi")])[0]

'Ailesinden birini kaybeden Ankaralılar, "Alo 188" Cenaze Merkezi\'ni ve ya Başkent 153’ü arayarak yardım talebinde bulunabilirler. Merkez, ölüm raporunun alınmasından dini vecibelerin yerine getirilmesine kadar tüm hizmetleri ücretsiz olarak gerçekleştirmektedir. Tüm nakiller Büyükşehir Belediyesi\'nin cenaze araçlarıyla yapılmaktadır. ALO 188 ekipleri, defin işleminin sonuna kadar vatandaşa destek sağlamaktadır.\n\nVefat olaylarında vatandaşların yapması gerekenler şunlardır:\n\n-Yaşanan bir ölüm olayında ALO 188 ücretsiz cenaze hattı aranarak bilgi verilir.\n-Kişi evde vefat etmişse ALO 188 ya da İlçe Belediyesi Cenaze Hizmetlerini arayarak , hastanede vefat etmiş ise hastanenin ilgili biriminden ölüm belgesi alınır.\n-Ölüm raporuyla vefat eden kişinin bağlı bulunduğu ilçe belediyesine müracaat edilir. Daha önce temin edilmiş mezar yeri yoksa veya mükerrer defin yapılmayacaksa Mezarlıklar Müdürlüğü ile görüşülmelidir.\n-Fakir ve kimsesiz kişilerden durumlarını belgelemeleri halinde 

In [35]:


Pipe.predict([metin_onisle("Öğrencilerin ücretsiz taşındığı hat")])[0]

'EGO Genel Müdürlüğü, Hacettepe Üniversitesi Beytepe Kampüsü’ne yoğunluğu sebebiyle ücretsiz 5 solo otobüs hizmeti vermeye başlamıştır. 21 körüklü otobüsün dışında ücretsiz 5 solo otobüs, haftanın 7 günü 06.30-20.00 saatleri arasında Beytepe Metrosu çıkışından Hacettepe Üniversitesi Beytepe Kampüs’ü içine kadar ücretsiz öğrenci taşımaktadır.'

In [ ]:
# sistem kendine update etsin, dışarıdan soru geli